- LOAN_ID
- PRINCIPAL_ORIGINATION
- PRINCIPAL_OUTSTANDING
- ORIGINATION_DATE
- CONTRACTUAL_MATURITY
- MATURITY_DATE
- INTEREST_RATE_TYPE (FIXED / FLOATING)
- INTEREST_RATE


### Simulation parameters

In [5]:
# Simulation parameters

# LOAN PORTFOLIO SIZE
NUM_LOANS = 5

# LOAN_ID

# PRINCIPAL_ORIGINATION
PRINCIPAL_ORIGINATION_MIN = 50 #in USD thousand
PRINCIPAL_OUTSTANDING_MAX = 500 #in USD thousand
PRINCIPAL_ORIGINATION_DISTRIBUTION = 'uniform'

# ORIGINATION_DATE
ORIGINATION_DATE_MIN = '2000-01-01'
ORIGINATION_DATE_MAX = '2020-09-30'

# CONTRACTUAL_MATURITY
CONTRACTUAL_MATURITY_MIN = 6 #in months
CONTRACTUAL_MATURITY_MAX = 30*12 #in months
CONTRACTUAL_MATURITY_DISTRIBUTION = 'uniform'

# MATURITY_DATE = ORIGINATION_DATE + CONTRACTUAL_MATURITY

# INTEREST_RATE_TYPE (FIXED / FLOATING)
FIXED_PROB = 0.4

# LOAN INTEREST_RATE
LOAN_INTEREST_RATE_FIXED_MIN = 0.03
LOAN_INTEREST_RATE_FIXED_MAX = 0.06

LOAN_INTEREST_RATE_FLOATING_MIN = 0.02
LOAN_INTEREST_RATE_FLOATING_MAX = 0.05

# MARKET INTEREST RATE
MARKET_INTEREST_RATE_INDEX = 0.01

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import *
import numpy_financial as npf

In [7]:
sns.set_style('whitegrid')

In [8]:
def random_dates(start_date, end_date): 

    start_u = start_date.value//(24*60*60*10**9)
    end_u = end_date.value//(24*60*60*10**9)
    return pd.to_datetime(np.random.randint(start_u, end_u, NUM_LOANS), unit='D')

In [9]:
def random_amount(distribution, min, max): 

    if distribution == 'normal':
        mu = (min + max) / 2
        sigma = (max - min) / 6
        return np.round(np.random.normal(mu, sigma, size=n),0)
    else:
        return np.random.randint(min, max, size=NUM_LOANS)
    

### Create loan IDs with unique alphanumeric series

In [10]:
list = []

for i in range(1, NUM_LOANS+1): 
    list.append(f'LOAN{i:05d}') 
df_loan_id = pd.DataFrame(data = list, columns = ["LOAN_ID"])
df_loan_id.head()

,LOAN_ID
0,LOAN00001
1,LOAN00002
2,LOAN00003
3,LOAN00004
4,LOAN00005


### Create original principal amounts

In [11]:
df_principal_origination = 1000*pd.DataFrame(
    data = random_amount(
        PRINCIPAL_ORIGINATION_DISTRIBUTION,
        PRINCIPAL_ORIGINATION_MIN,
        PRINCIPAL_OUTSTANDING_MAX),
    columns = ["Origination_Principal"])
df_principal_origination.head()

,Origination_Principal
0,391000
1,474000
2,213000
3,167000
4,108000


### Create loan origination dates

In [12]:
dates = random_dates(pd.to_datetime(ORIGINATION_DATE_MIN), pd.to_datetime(ORIGINATION_DATE_MAX))
df_dates = pd.DataFrame({'Origination_Dates': dates})
df_dates.head()

,Origination_Dates
0,2016-10-13
1,2018-12-12
2,2007-10-06
3,2017-10-23
4,2003-01-25


### Create loan contractual maturity

In [13]:
df_contractual_maturity = pd.DataFrame(
    data = random_amount(
        CONTRACTUAL_MATURITY_DISTRIBUTION,
        CONTRACTUAL_MATURITY_MIN,
        CONTRACTUAL_MATURITY_MAX),
    columns = ["Contractual_Maturity"])
df_contractual_maturity.head()

,Contractual_Maturity
0,162
1,135
2,318
3,272
4,155


### Create interest rate type, that is, fixed or floating

In [16]:
interest_type = np.random.choice(a=['Fixed', 'Floating'], size=NUM_LOANS, p=[FIXED_PROB, (1 - FIXED_PROB)])
df_interest_rate = pd.DataFrame(interest_type, columns = ['Fixed_Floating'])


df_interest_rate.head()

,Fixed_Floating
0,Floating
1,Fixed
2,Floating
3,Fixed
4,Fixed


In [17]:
df_interest_rate['Interest_Rate_Fixed'] = df_interest_rate['Fixed_Floating'].apply(lambda x: np.random.uniform(LOAN_INTEREST_RATE_FIXED_MIN*100, LOAN_INTEREST_RATE_FIXED_MAX*100) if x == 'Fixed' else 'N/A')
df_interest_rate['Interest_Rate_Floating_Index'] = df_interest_rate['Fixed_Floating'].apply(lambda x: 'Index' if x == 'Floating' else 'N/A')
df_interest_rate['Interest_Rate_Floating_Spread'] = df_interest_rate['Fixed_Floating'].apply(lambda x: np.random.uniform(LOAN_INTEREST_RATE_FLOATING_MIN*100, LOAN_INTEREST_RATE_FLOATING_MAX*100) if x == 'Floating' else 'N/A')
df_interest_rate.head()

,Fixed_Floating,Interest_Rate_Fixed,Interest_Rate_Floating_Index,Interest_Rate_Floating_Spread
0,Floating,N/A,Index,3.9944
1,Fixed,5.49022,N/A,N/A
2,Floating,N/A,Index,2.89995
3,Fixed,4.09528,N/A,N/A
4,Fixed,3.67361,N/A,N/A


### Cobining all characteristics to creatre the dataframe

In [18]:
df = pd.concat(
    [
        df_loan_id,
        df_dates,
        df_principal_origination,
        df_contractual_maturity,
        df_interest_rate
    ], axis = 1)
df.head()

,LOAN_ID,Origination_Dates,Origination_Principal,Contractual_Maturity,Fixed_Floating,Interest_Rate_Fixed,Interest_Rate_Floating_Index,Interest_Rate_Floating_Spread
0,LOAN00001,2016-10-13,391000,162,Floating,N/A,Index,3.9944
1,LOAN00002,2018-12-12,474000,135,Fixed,5.49022,N/A,N/A
2,LOAN00003,2007-10-06,213000,318,Floating,N/A,Index,2.89995
3,LOAN00004,2017-10-23,167000,272,Fixed,4.09528,N/A,N/A
4,LOAN00005,2003-01-25,108000,155,Fixed,3.67361,N/A,N/A


### Include calculated fields

- Maturity dates
- Interest rates
- Loan status (active / closed)

In [19]:
df['Maturuty_Dates'] = df['Origination_Dates'] + df['Contractual_Maturity'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))

df.head()

,LOAN_ID,Origination_Dates,Origination_Principal,Contractual_Maturity,Fixed_Floating,Interest_Rate_Fixed,Interest_Rate_Floating_Index,Interest_Rate_Floating_Spread,Maturuty_Dates
0,LOAN00001,2016-10-13,391000,162,Floating,N/A,Index,3.9944,2017-03-24
1,LOAN00002,2018-12-12,474000,135,Fixed,5.49022,N/A,N/A,2019-04-26
2,LOAN00003,2007-10-06,213000,318,Floating,N/A,Index,2.89995,2008-08-19
3,LOAN00004,2017-10-23,167000,272,Fixed,4.09528,N/A,N/A,2018-07-22
4,LOAN00005,2003-01-25,108000,155,Fixed,3.67361,N/A,N/A,2003-06-29


In [20]:
# df['Principal_Outstanding'] = npf.fv(df['Interest_Rate']/12, )